# Using Genetic Algorithms to Form Optimal Groups

We have all been in that situation. You have to do a group project in a class, and the lecturer assigns the groups randomly. You find yourself in a group with people you don’t know and end up doing all the work by yourself. We want to change that and want you to implement an Genetic Algorithm that optimizes groups.


## Task 1:


You have to implement the initial population, crossover, mutation, and fitness function. We divided it into smaller subtasks for you and implemented some of the functions for you. If you want to do your own implementation, feel free to ignore our hints and functions.

Your initial population consists of 50 group distributions, where 100 students are assigned to 20 groups. Each student has a Student ID, a name, their spoken language, their 2 Majors (we assume everyone is doing their Masters), their ambition in the course, their preferred meeting place, their personality type, their gender, a friend that they want to be in a group with, and their preferred meeting day.

To get you even more involved with the task, we want every member of your group to add their own person into the data. To do so, you simply have to execute the Jupyter Notebook named "Dataset_Input.ipynb". We are using the Myers-Briggs personality, which may not be the most scientific, but it for sure is entertaining. If you don’t know your type, you can take the test here (approx. 10 minutes) https://www.16personalities.com/free-personality-test. 

Run this cell to load the packages:

In [2]:
import numpy as np
import random
import pandas as pd
import matplotlib as plt
import ipywidgets
import IPython
from Generator import DataGenerator
from typing import List

Execute this cell to create the dataset:

In [3]:
generator = DataGenerator()

#df, type_ = generator.create_basic()

df, type_ = generator.create_full()

df.to_csv("dataset_" + type_ + ".csv", encoding="utf-8-sig", index=False)

Please provide the number of participants: 10


Hyperparameters:

In [4]:
students: pd.DataFrame = pd.read_csv("dataset_full.csv")
student_ids: List[int] = students.ID.tolist()

# hyperparameters
num_individuals: int = 100
groupsize: int = 5
# between 0 and 1
mutation_rate: float = 0.05

students.tail()

,ID,Name,Gender,Preferred language,Majors,Level of ambition,Preferred meeting place,Personality type,Best friend,Preferred day
5,6,Christin Foerster,Female,Any,"('CP', 'NI')",Very low,In person,ESFP,Marco Loewe,Wednesday
6,7,Nicole Fiedler,Female,English,"('CL', 'CP')",High,In person,ENTP,Barbara Schuhmacher,Monday
7,8,Kevin Schröder,Male,Any,"('AI', 'NI')",Low,In person,ESFJ,Sabrina Reinhardt,Wednesday
8,9,Jan Wulf,Male,Any,"('NS', 'AI')",Medium,In person,ESFJ,Jessika Austerlitz,Friday
9,10,Andreas Jaeger,Male,Any,"('NS', 'CP')",Very low,In person,ENTJ,Uwe Bader,Friday


We create one random individual:

In [5]:
def create_random_individual(student_ids: List[int]) -> List[int]:
    #You don't need to do anything here.

    individual = student_ids.copy()
    random.shuffle(individual)

    return individual

print(create_random_individual(student_ids))

[7, 2, 5, 4, 8, 10, 6, 1, 9, 3]




Create the initial population of 50 (=num_individuals) individuals:


In [6]:
def create_initial_population(students: pd.DataFrame, groupsize: int) -> np.ndarray:
    
    return
    np.array([create_random_individual(student_ids)] for n in range(allstudents))

    #should return a numpy array of the whole population

We need a fitness function that computes how good the group distribution is. You have to take into consideration that we have many parameters that are not equally important and should therefore be differently weighted. We already coded some of the specific evaluation functions for the parameters for you. You have to implement the remaining evaluation functions. Think about what is desirable to have in a group and how you can calculate it. For example, you might have to change the data type of a parameter to make meaningful calculatons.

At the end, you have to weigh all evaluation functions in one fitness function. 

In [95]:
# if everyone prefers the same language return 5, else return 0
def evaluate_language(group: pd.DataFrame) -> float:
    language = group['Preferred language'].value_counts()
    
    if language[0] == groupsize or language[0] + language[1] == groupsize or language[0] + language[2] == groupsize:
        return 5
    
    return 0

In [96]:
# the more majors the better, to collect all the knowledge
# if every major is present return 5, else return the share of 5
def evaluate_majors(group: pd.DataFrame) -> float:
    majors = group['Majors'];
    group_majors = [];
    for individual_majors in majors:
        for major in eval(individual_majors):
            if major not in group_majors:
                group_majors.append(major)
    return len(group_majors)/7*5

In [97]:
# keep people with similar ambition together so no one has to pull others through
def evaluate_ambition(group: pd.DataFrame) -> float:
    
    ambitions = group['Level of ambition'].value_counts()
    score = 0
    vhigh = ambitions['Very high']
    high = ambitions['High']
    medium = ambitions['Medium']
    low = ambitions['Low']
    vlow = ambitions['Very low']
    
    if vhigh == groupsize or high == groupsize or medium == groupsize or low == groupsize or vlow == groupsize:
        return 5
    elif vhigh+high == groupsize or high+medium == groupsize or medium+low == groupsize or low+vlow == groupsize:
        return 4
    elif vhigh+high+medium == groupsize or high+medium+low == groupsize or medium+low+vlow:
        return 3
    elif vhigh+high+medium+low == groupsize or high+medium+low+vlow:
        return 2
    else:
        return 0

In [98]:
def evaluate_meeting_place(group: pd.DataFrame) -> float:
    #You don't need to code anything in here.
    #This is an example evaluation function.
    
    # number of groupmembers for each preferred meeting place
    meeting_place = group['Preferred meeting place'].value_counts()

    # if all prefer the same meeting place return 5, else 0
    if meeting_place[0] == groupsize:
        return 5

    return 0

In [99]:
# keep the gender balanced
def evaluate_gender(group: pd.DataFrame) -> float:
    gender = group['Gender'].value_counts()
    # get the number of differences between participants of each gender & add them
    dif = gender[0] - gender[1] + gender[0] - gender[2] + gender[1] - gender[2]
    # 3 gender so max dif if everyone has the same is groupsize*2, subtract dif to punish same sex groups & divide by
    # groupsize * multiply by five to get max val 5
    return (groupsize*2-dif)/(groupsize*2)*5

In [100]:
# return 5 if every friend is present, if one is not present the value goes down to 0 when no friend is present
def evaluate_friends(group: pd.DataFrame) -> float:
    friends = group['Best friend']
    maxval = 5
    for friend in friends:
        if friend not in group['Name'].values:
            maxval = maxval - (5/groupsize)
    return maxval

In [101]:
def evaluate_personality(group: pd.DataFrame) -> float:
    #You dont need to change anything in here 

    #information about compatible personality types is taken from
    # Montequín, Vicente Rodríguez, et al. "Using Myers-Briggs type indicator (MBTI) as a tool for setting up student teams for information technology projects." Journal of Information Technology and Application in Education 1.1 (2012): 28-34.

    #count existing personality types in each group
    personalities = group['Personality type']
    types = personalities.value_counts()

    #fitness function starts with 0 and gets better
    # with every good group member
    fitness = 0

    #its good if there is a group leader like an ISTJ or an ESTJ, but only one
    try:
        if (types['ISTJ'] + types['ESTJ'] == 1):
            fitness+=5
        elif (types['ISTJ'] + types['ESTJ'] >= 2):
            fitness-=5
    except KeyError:
        pass

    #compare compatibility of group members
    for i, personality_a in enumerate(personalities.tolist()):
        for j, personality_b in enumerate(personalities.tolist()):
            # skip same group member and members already compared
            if i <= j:
                continue

            # increase fitness if
            if (personality_a[1] != personality_b[1]) ^ (personality_a[2] != personality_b[2]):
                if (personality_a[0] != personality_b[0]) or (personality_a[3] != personality_b[3]):
                    fitness+=1

    return fitness

In [102]:
# return 5 if everyone prefers the same day, else return 0
def evaluate_meeting_day(group: pd.DataFrame) -> float:
    meeting_day = group['Preferred day'].value_counts();
    if meeting_day[0] == groupsize:
        return 5

    return 0

And now put everything together.
The function is almost done, but remember to add the weights.

In case you want to add hard constraints, feel free to do that in this function.



In [111]:
def evaluate_fitness(individual: np.ndarray, students: pd.DataFrame):
    # split individual into student groups of the groupsize
    groups = np.array_split(individual, (len(individual)/groupsize))

    # iterate over groups and calculate scores for the different parameters
    scores = []
    for group_ids in groups:
        # get full data for students in this group from pd dataframe
        group = students.loc[students['ID'].isin(group_ids)]

        # get individual scores for parameters
        # language is most important in my view, followed by meeting day, place and ambition, so I weighted accordingly
        language_score = 4 * evaluate_language(group)
        major_score = evaluate_majors(group)
        ambition_score = 2 * evaluate_ambition(group)
        place_score = 3 * evaluate_meeting_place(group)
        gender_score = evaluate_gender(group)
        friend_score = evaluate_friends(group)
        personality_score = evaluate_personality(group)
        day_score = 3 * evaluate_meeting_day(group)

        # formula for adding and weighting different scores
        scores.append(language_score+major_score+ambition_score+place_score+gender_score+friend_score+personality_score+day_score)

    #Convert to series to calculate mean more easily
    return pd.Series(scores).mean()
    

Now, you have to code a crossover function, which takes 2 individuals based on their fitness function and produces a child from them. Use the tournament selection for parent selection.
For the crossover, we want you to use the uniform crossover function with random templates. 

In [112]:
def tournament_selection(population: np.ndarray, tournament_size: int) -> List[np.ndarray]:
    

    #should return 2 parents as a list
    pass


Use a boolean template in the length of the individual, this can be hardcoded or generated randomly every time to add more variance. On the places where the template is true use the genes from parent1, then take all the genes from parent2 that are not used and add them to the empty places in the child in the same order as they appear on parent2

In [113]:
def uniform_order_crossover(parent1: np.ndarray, parent2: np.ndarray, template: np.ndarray) -> np.ndarray:
    pass

The last thing you need is the mutation function. It should take the individual produced by the crossover function and mutate it with a chance of for example 5%. A mutation switches the assigned groups of 2 people. 

In [114]:
def mutation(individual: np.ndarray, mutation_rate: float) -> np.ndarray:
    pass


You can now execute the code below and see if everything is working.

In [115]:
#episodes is the number of episodes after the algorithm stops
#num_replace is the number of unfit individuals that will be replaced
def genetic_algorithm(episodes: int, num_replace: int) -> np.ndarray:
    
    
    
    #visualization:
    pass

In [116]:
population = genetic_algorithm(20,2)

### Let's find your group or the one of a friend

In [143]:
def get_groups_from_individual(individual: np.ndarray) -> List[np.ndarray]:
    """ Returns a Python List of all Groups in an individual(as numpy ndarrays) """
    groups = pd.DataFrame()
    for i in individual:
        groups = pd.concat([groups, students.loc[students['ID'] == i]])

    groups = groups.reset_index(drop=True)
    nested_groups_list = []
    for n,j in enumerate(range(0, len(groups), groupsize)):
        nested_groups_list.append(pd.DataFrame(groups[j:j+groupsize]))
    return nested_groups_list

def get_groups_by_person_fullname(fullname: str, individual: np.ndarray) -> List[np.ndarray]:
    """ Returns all Groups that got a Person with $fullname$ in given $individual$"""
    nested_groups_list = get_groups_from_individual(individual)

    groups_with_person = []
    for group in nested_groups_list:
        if fullname in group["Name"].values:
            groups_with_person.append(group)
    return groups_with_person

def get_group_by_person_ID(ID: int, individual: np.ndarray) -> np.ndarray:
    """ Returns Group that has Person with given $ID$ in given $individual$"""
    nested_groups_list = get_groups_from_individual(individual)
    groups_with_person = []
    for group in nested_groups_list:
        if ID in group["ID"].values:
            return group
    return None


#### Find your group by your Fullname

In [145]:
######### ATTENTION ##########
# Define your individual here!!! At the moment it will take a random individual! Pick for example the fittest individual.
individual = create_random_individual(student_ids)

# User Input for the Full Name - Please be accurate!
fullname = str(input("Please the Full-Name you want to look up: "))
# Get the group by your Full Name - If there are multiple Persons with your name you will get all of them
my_group = get_groups_by_person_fullname(fullname, individual)
# Plotting the first group found
if my_group:
    print(my_group)
else:
    print("Didn't find the requested Person with given Fullname")

[     ID             Name  Gender Preferred language          Majors  \
75   48    Torsten Beyer  Female                Any  ('CP', 'PHIL')   
76   72     Karolin Bach  Female                Any  ('CL', 'PHIL')   
77   38   Heike Schwartz  Female                Any    ('NI', 'NS')   
78  101      Donald Dump  Divers                Any    ('AI', 'NI')   
79   61  Leon Zimmermann    Male            English  ('AI', 'PHIL')   

   Level of ambition Preferred meeting place Personality type  \
75              High               In person             INTP   
76               Low               In person             ENFJ   
77               Low               In person             INFP   
78            Medium                  Online             ENTP   
79               Low               In person             ENTJ   

             Best friend Preferred day  
75  Florian Baumgaertner      Thursday  
76     Michael Eberhardt     Wednesday  
77        Karin Glockner      Thursday  
78           Dani

#### Find your group by your ID

In [146]:
# Find your group by your ID
my_group = get_group_by_person_ID(101, individual)
# Plotting the group, if one found with given ID
if my_group is not None:
    print(my_group)
else:
    print("Didn't find the requested Person with given ID")

     ID             Name  Gender Preferred language          Majors  \
75   48    Torsten Beyer  Female                Any  ('CP', 'PHIL')   
76   72     Karolin Bach  Female                Any  ('CL', 'PHIL')   
77   38   Heike Schwartz  Female                Any    ('NI', 'NS')   
78  101      Donald Dump  Divers                Any    ('AI', 'NI')   
79   61  Leon Zimmermann    Male            English  ('AI', 'PHIL')   

   Level of ambition Preferred meeting place Personality type  \
75              High               In person             INTP   
76               Low               In person             ENFJ   
77               Low               In person             INFP   
78            Medium                  Online             ENTP   
79               Low               In person             ENTJ   

             Best friend Preferred day  
75  Florian Baumgaertner      Thursday  
76     Michael Eberhardt     Wednesday  
77        Karin Glockner      Thursday  
78           Danie

## Task 2

Play around with the different values like initial population size, mutation rate, fitness function, and number of students and observe when it works the best. Write your insights down here.